In [18]:
import os
import re
import pandas as pd
import requests
import urllib.request
import pickle
from tqdm import tqdm
from datetime import datetime

from bs4 import BeautifulSoup
import warnings
warnings.simplefilter('once') # Comment out to see warnings if needed

In [2]:
# Parent Directoy URL
url= "http://reports.ieso.ca/public/Demand/"
page = requests.get(url,allow_redirects=True)

In [3]:
# Retrieve File names from parent directory url
soup = BeautifulSoup(page.text)
xml_cap=soup.find_all("a",href=True)
files=[]
for element in xml_cap:
    file_name=element.text
    if re.search("\.csv$", file_name) and not re.search("_v", file_name):
        files.append(str(file_name))

In [2]:
# Setting up GCloud Storage Authentication
from google.cloud import storage
path_to_private_key = './ieso-dashboard-c639f1a39298.json'
client = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)
bucket = client.bucket('amol_javahire')

In [5]:
# Downloading the csv files as it is
# warnings.simplefilter('once') # Comment out to see warnings if needed

## LOCAL DATA CURATION

# outdir = './Demand'
# if not os.path.exists(outdir):
#     os.mkdir(outdir)
# for file in files:
#     print('Processing : ' + file)
#     curr_url=str(url)+str(file)
#     fullname = os.path.join(outdir, (str(file)))
#     urllib.request.urlretrieve(curr_url, fullname)
# print('Downloaded all csv files in ./Demand')

In [9]:
pd.set_option('display.max_columns', None)
pd.reset_option('max_columns')

In [33]:
## GCP DATA CURATION

# Defining Output Data Structure and extracting and filling values correspondingly and then dump final data to ./[outdir]
col_names=['mkt_date','mkt_he','mkt_demand','ont_demand']

for file in tqdm(files,desc="Processing :: "):
    curr_url=str(url)+str(file)
    df=pd.read_csv(curr_url,names=col_names,skiprows=4,header=None)
    created_at=pd.read_csv(curr_url,nrows=2).iloc[0,0]
    
    # Regex to capture only the date from string
    match_str = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', created_at)
    date_val = datetime.strptime(match_str.group(), '%Y-%m-%d %H:%M:%S').isoformat()

    # Inserting  created_at column
    df.insert(0,'created_at', date_val)

    blob = bucket.blob('Demand/'+str(file))
    blob.upload_from_string(df.to_csv(index=False), 'text/csv')
print('Dumped all csv files in ./Demand')

Processing :: 100%|█████████████████████████████| 22/22 [01:43<00:00,  4.73s/it]

Dumped all csv files in ./Demand


In [6]:
# pd.set_option('display.max_columns', None)
# pd.reset_option(‘max_columns’)

In [4]:
from google.cloud import bigquery

# Construct a BigQuery client object.
path_to_private_key = './ieso-dashboard-c639f1a39298.json'
client = bigquery.Client.from_service_account_json(json_credentials_path=path_to_private_key)

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "ieso-dashboard.Demand.test"

job_config = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
uri = "gs://amol_javahire/Demand/PUB_Demand_*"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

LoadJob<project=ieso-dashboard, location=US, id=c4ed8c3d-e39a-4428-a4cb-c751c803bd6d>